In [5]:
%load_ext autoreload
%autoreload 2

from notebook import *

In [1]:
# # Copie les notebooks et supprime les sorties.
# # Décommenter et exécuter cette cellule avant de commiter/pusher
# # les modifications du notebook sur Github.
# copy_and_clean_notebooks()

# Introduction

Ce notebook va nous permettre de mettre à jour le jeu de données de LUIS et de mettre à jour les paramètres de LUIS en conséquence. Nous allons notamment ajouter dans le jeu de données les textes extraits des analyses d'insatisfactions.

<img src="./data/img/archi_update_luis.png" alt="Mise à jour de LUIS" width="700"/>
<p style="text-align: center; text-decoration: underline;">Mise à jour de LUIS</p>

Attention de ne pas exécuter ce notebook après chaque analyse des insatisfactions. D'après la documentation de LUIS ([en savoir plus](https://docs.microsoft.com/en-us/azure/cognitive-services/luis/luis-concept-best-practices#dont-train-and-publish-with-every-single-example-utterance)), il est recommendé d'avoir au moins 15 nouveaux textes à ajouter au jeu de données pour observer des améliorations du modèle.

# Création d'une nouvelle branche git

Commencer par créer une nouvelle branche git pour effectuer et tester vos modifications.

# Chargement des ressources

Nous allons charger toutes les ressources Azure qui vont nous permettre de créer et d'enregistrer des jeux de données.

Nous allons aussi charger les paramètres du modèle LUIS actuel.

## Chargement du workspace

In [6]:
# On charge l’espace de travail Azure Machine Learning existant
ws = Workspace.from_config()

## Chargement du magasin des données

In [7]:
# On charge le magasin de données par défaut
datastore = ws.get_default_datastore()

## Chargement des paramètres de LUIS

In [8]:
# On charge les variables d'environnement de LUIS
env = LUISEnv("../P10_02_luis/.env")

In [9]:
# On charge les paramètres du modèle
with open("../P10_02_luis/params.json") as f:
    params = json.load(f)

# Mise à jour du jeu de données

Dans cette partie, nous allons labelliser de nouvelles données et les ajouter au jeu de données existant.

## Chargement des textes à labelliser

### Intent `book_flight`

Copier dans la liste ci-dessous les textes ayant pour intent `book_flight` :

In [11]:
# On colle les nouveaux textes à labelliser
book_flight_texts = [
    "Book me a flight from London to Paris tomorrow. I have only 100€."
]

len(book_flight_texts)

1

### Intent `None`

Copier dans la liste ci-dessous les textes ayant pour intent `None` :

In [12]:
# On colle les nouveaux textes à labelliser
none_texts = [
    "Hey !!!"
]

len(none_texts)

1

## Chargement du précédent jeu de données

On va récupérer le jeu d'entrainement et le jeu de test du dataset utilisé pour entrainer le modèle actuel.

In [13]:
# On récupère les information du dataset
ds_name = params["dataset"]["name"]
ds_version = params["dataset"]["version"]

In [14]:
with tempfile.TemporaryDirectory() as tmp_dir_name:
    dataset = Dataset.get_by_name(ws, **params["dataset"])
    dataset.download(target_path=tmp_dir_name, overwrite=False)

    # On charge le jeu d'entrainement
    file_path = os.path.join(tmp_dir_name, "utterances_train.json")
    with open(file_path) as f:
        utterances_train = json.load(f)

    # On charge le jeu de test
    file_path = os.path.join(tmp_dir_name, "utterances_test.json")
    with open(file_path) as f:
        utterances_test = json.load(f)

## Suppression des doublons

On va extraire les textes de ces jeux de données afin de supprimer les doublons dans les nouveaux textes à labelliser :

In [15]:
# Extraction des textes du jeu de données pour l'intent "book_flight"
old_texts = get_texts_from_dataset(
    utterances_train,
    utterances_test,
    "book_flight"
)

# Extraction des textes du jeu de données pour l'intent "None"
old_texts += get_texts_from_dataset(
    utterances_train,
    utterances_test,
    "None"
)

### Intent `book_flight`

On supprime les doublons pour l'intent `book_flight`.

In [16]:
# On supprime les doublons
book_flight_texts = [i for i in book_flight_texts if i not in old_texts]

len(book_flight_texts)

1

### Intent `None`

On supprime les doublons pour l'intent `None`.

In [17]:
# On supprime les doublons
none_texts = [i for i in none_texts if i not in old_texts]

len(none_texts)

1

## Transformation des données

Nous allons maintenant convertir nos textes pour les mettre au format LUIS.

### Intent `book_flight`

In [18]:
# On convertit les textes au format LUIS
new_utterances = texts_to_luis_utterances(book_flight_texts, "book_flight")

### Intent `None`

In [19]:
# On convertit les textes au format LUIS
new_utterances += texts_to_luis_utterances(none_texts, "None")

## Labellisation des nouveaux textes

Afin de labelliser les nouveaux textes, nous allons utiliser l'outil de labellisation de LUIS Portal ([en savoir plus](https://docs.microsoft.com/en-us/azure/cognitive-services/luis/sign-in-luis-portal)).

### Création d'un modèle LUIS pour la labellisation

Nous allons commencer par créer une version du modèle actuel sans aucun texte :

In [20]:
# Nom de version spécial pour la labellisation
labellisation_app_version = "labellisation"

In [21]:
# On crée la nouvelle version
create_new_version(env, labellisation_app_version, params["model"], new_utterances)

### Labellisation manuelle

La labellisation des nouveaux textes doit se faire manuellement ([en savoir plus](https://docs.microsoft.com/en-us/azure/cognitive-services/luis/label-entity-example-utterance)) :
- Aller sur LUIS Portal.
- Dans la section `MANAGE/Versions` sélectionner la version `labellisation` et cliquer sur `Activate` pour l'activer.
- Aller ensuite dans la section `BUILD/Intents` et sélectionner l'intent `book_flight`.
- Labelliser les textes de l'intent en sélectionnant les entities et en leur attribuant un label.

<img src="./data/img/data_labellisation.png" alt="Exemple de labellisation d'une phrase sur LUIS Portal" width="700"/>
<p style="text-align: center; text-decoration: underline;">Exemple de labellisation d'une phrase sur LUIS Portal</p>

### Téléchargement des utterances labellisées

Une fois les textes labellisés sur LUIS Portal, il nous suffit de les télécharger :

In [22]:
new_utterances = get_utterances(env, labellisation_app_version)

Observons la première donnée afin de vérifier que la labellisation a bien été effectuée :

In [25]:
pprint_dict(new_utterances[0])

{
  "text": "book me a flight from london to paris tomorrow. i have only 100\u20ac.",
  "intent": "book_flight",
  "entities": [
    {
      "entity": "from_city",
      "startPos": 22,
      "endPos": 27,
      "children": []
    },
    {
      "entity": "to_city",
      "startPos": 32,
      "endPos": 36,
      "children": []
    },
    {
      "entity": "from_dt",
      "startPos": 38,
      "endPos": 45,
      "children": []
    },
    {
      "entity": "budget",
      "startPos": 60,
      "endPos": 63,
      "children": []
    }
  ]
}


In [44]:
# On ssuprime la version temporaire de labellisation
delete(env, labellisation_app_version)

## Split des données

Nous allons séparer nos données en un jeu d'entrainement et un jeu de test.

In [26]:
# On va prendre 70% des données pour le jeu d'entrainement
train_nb = int(len(new_utterances) * 0.7)

In [27]:
# On mélanges les utterances
random.shuffle(new_utterances)

In [28]:
# On crée le jeu d'entrainement
new_utterances_train = new_utterances[:train_nb]

In [29]:
# On crée le jeu de test
new_utterances_test = new_utterances[train_nb:]

## Ajout des précédentes utterances

On ajoute les nouveaux textes labellisés au précédents jeux de données :

In [30]:
# On ajoute des nouveaux textes au jeu d'entrainement
utterances_train += new_utterances_train

In [31]:
# On ajoute des nouveaux textes au jeu de test
utterances_test["LabeledTestSetUtterances"] += new_utterances_test

## Enregistrement des datasets

Nous allons enregistrer nos données au format JSON dans le Datastore :

In [35]:
with tempfile.TemporaryDirectory() as tmp_dir_name:
    # On enregistre les données
    file_path = os.path.join(tmp_dir_name, "utterances_train.json")
    with open(file_path, "w") as f:
        json.dump(utterances_train, f)
        
    file_path = os.path.join(tmp_dir_name, "utterances_test.json")
    with open(file_path, "w") as f:
        json.dump(utterances_test, f)
    
    # On upload tous les fichiers dans le datastore
    ds = Dataset.File.upload_directory(
        tmp_dir_name,
        target=(datastore, "utterances/" + datetime.now().strftime("%Y_%m_%d_%H_%M_%S")),
        overwrite=True,
        show_progress=True
    )

Validating arguments.
Arguments validated.
Uploading file to utterances/2022_01_02_13_47_28
Uploading an estimated of 2 files
Uploading /tmp/tmpthtrkpxe/utterances_test.json
Uploaded /tmp/tmpthtrkpxe/utterances_test.json, 1 files out of an estimated total of 2
Uploading /tmp/tmpthtrkpxe/utterances_train.json
Uploaded /tmp/tmpthtrkpxe/utterances_train.json, 2 files out of an estimated total of 2
Uploaded 2 files
Creating new dataset


On crée ensuite un Dataset Azure à partir de ces fichiers :

In [36]:
ds = ds.register(
    workspace=ws,
    name="utterances",
    description="Train and test utterances",
    create_new_version=True
)

In [37]:
print(f"Création du dataset '{ds.name}' version {ds.version}.")

Création du dataset 'utterances' version 2.


# Enregistrement des paramètres de LUIS sur Github

Il existe plusieurs façons de créer un modèle LUIS. Nous avons d'abord testé le tutorial suivant qui utilise le SDK Python : [quickstart](https://docs.microsoft.com/en-us/azure/cognitive-services/luis/client-libraries-rest-api?tabs=windows&pivots=programming-language-python).

Nous nous sommes ensuite apperçut que l'on pouvait importer et exporter notre modèle au format JSON ([en savoir plus](https://docs.microsoft.com/en-us/azure/cognitive-services/luis/app-schema-definition)). Ce format s'avère pratique pour versionner le modèle et il reste assez simple pour être manipulé dans le cadre de recherche d'hyperparamètres. Cette méthode utilise le SDK python ainsi que l'API REST de LUIS. Les paramètres du modèle LUIS ainsi que les informations du jeu de données sont stockés dans le fichier `../P10_02_luis/params.json`. C'est ce fichier qui sera ensuite utilisé par les Github actions pour déployer le modèle en production.

## Mise à jour des paramètres de LUIS

On va donc commencer par mettre à jour le fichier JSON, notamment les informations du jeu de données précédemment créé.

In [38]:
new_params = params.copy()

In [39]:
# # En cas de modification du modèle LUIS, penser à incrémenter son numéro de version
# new_model_version = float(model_version) + 0.1
# new_model_version = f"{new_model_version:0.1f}"

# new_params["model"]["versionId"] = new_model_version

In [40]:
# On met à jour les paramètres avec le jeu de données que l'on a créé
new_params["dataset"]["name"] = ds.name
new_params["dataset"]["version"] = ds.version

In [41]:
# On enregistre les nouveaux paramètres
with open("../P10_02_luis/params.json", "w") as f:
    json.dump(new_params, f)

## Evaluation du nouveau modèle

Nous allons utiliser les mêmes briques logicielles que celles utilisées pour le déployement du modèle en production. Elles sont disponible dans le fichier `../P10_02_luis/utils.py`.

In [42]:
# On commence par créer un nom de version temporaire
tmp_app_version = "tmp"

In [45]:
# On crée ensuite une nouvelle version du modèle.
# Un premier modèle a déjà été créé dans le script de création
# des ressources de LUIS "../P10_02_luis/luis_create.sh".
# Ce 1er modèle a été créé à partir du fichier "../P10_02_luis/params.json".
create_new_version(env, tmp_app_version, new_params["model"], utterances_train)

In [46]:
# On entraine notre modèle
train(env, tmp_app_version)

In [47]:
# On le déploie sur l'environnement de test
deploy(env, tmp_app_version, "staging")

In [49]:
# On évalue le modèle avec le jeu de test
res = evaluate(env, is_staging=True, utterances=utterances_test)
res

,model_name,model_type,precision,recall,f_score
0,book_flight,Intent Classifier,0.92,1.00,0.96
1,None,Intent Classifier,1.00,0.79,0.88
2,from_dt,Entity Extractor,0.86,0.90,0.88
3,to_dt,Entity Extractor,0.88,1.00,0.93
4,budget,Entity Extractor,0.88,1.00,0.93
5,from_city,Entity Extractor,0.57,0.85,0.69
6,to_city,Entity Extractor,0.50,1.00,0.67


Pour chaque intent et chaque entity, on obtient 3 scores :
- `precision` : parmis les labels prédit sur chaque mot, indique lesquels sont corrects.
- `recall` : parmis les labels à détecter, indique lesquels ont été détéctés par le modèle.
- `f_score` : moyenne harmonique de la precision et du recall.

Pour un premier modèle, on obtient des scores satisfaisants.

On remarque que le recall de l'intent `None` est à 0.78. Le modèle semble donc louper certain de ces intents.

On s'apperçoit aussi que la precision des entités `from_city` et `to_city` sont à 0.57 et 0.5. Le modèle semble donc créer beaucoup de faux positifs sur ces entités.

In [50]:
# On supprime la version temporaire
delete(env, tmp_app_version)

## Enregistrement du Github

Commiter et pusher sur Github les modifications du fichier `../P10_02_luis/params.json`.

Effectuer ensuite une demande de Pull request afin que ces nouveaux paramètres puissent êtres utilisés lors du déploiement du modèle.